In [2]:
#TRATAR EL TEXTO Y GUARDAR EN UN PKL:
from pickle import load
from pickle import dump
from numpy.random import shuffle
import string
import re
from unicodedata import normalize
from numpy import array
import numpy as np
from numpy import argmax
#KERAS PARA EL MODELO:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
#PARA CARGAREL MODELO:
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

Using TensorFlow backend.


In [3]:
# carga doc en la memoria
def cargar_doc(filename):
    # abrir el archivo como de sólo lectura
    file = open(filename, mode='rt', encoding='utf-8')
    # leer todo el texto
    text = file.read()
    # cerrar el archivo
    file.close()
    return text

In [4]:
# dividir un documento cargado en oraciones
def crea_pares(doc):
    #Separa cada salto de linea:
    lines = doc.strip().split('\n')
    #Separa cada que se encuentre un tabulador en una lista otra lista con 
    #el texto en ingles y en italiano en tipo pares:
    pares = [line.split('\t') for line in lines]
    return pares

In [5]:
# limpiar una lista de líneas
def limpiar_pares(lines):
    cleaned = list()
    # preparar a regex para el filtrado de caracteres
    re_punc = re.compile('[%s]'% re.escape(string.punctuation))
    re_print = re.compile('[^%s]'% re.escape(string.printable))
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalizar los caracteres unicode
            line = normalize('NFD', line).encode('ascii','ignore')
            line = line.decode('UTF-8')
            # tokenize en espacio en blanco
            line = line.split()
            # convertir a minúsculas
            line = [word.lower() for word in line]
            # eliminar la puntuación de cada token
            line = [re_punc.sub('', w) for w in line]
            # eliminar caracteres no imprimibles de cada token
            line = [re_print.sub('', w) for w in line]
            # eliminar tokens con números en ellos
            line = [word for word in line if word.isalpha()]
            # store como cadena
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [6]:
#guardar una lista de oraciones limpias para archivar
def guarda_datos_limpios(lineas, filename):
    dump(lineas, open(filename,'wb'))
    print('Saved: %s'% filename)

In [7]:
#Cargar un conjunto de datos limpio
def cargar_datos_limpios(filename):
    return load(open(filename,'rb'))

In [9]:
#DATOS DE ENTRENAMIENTO:
#Cargar conjunto de datos de entrenamiento:
texto = cargar_doc(r'C:\Users\fernando\Desktop\PLN Traductor\Final3\data3.test')
texto2 = cargar_doc(r'C:\Users\fernando\Desktop\PLN Traductor\Final3\data3.eval')
#Dividirlo en parejas ingles-italiano:
pares = crea_pares(texto)
pares2 = crea_pares(texto2)
#Limpiar esas oraciones:
pares_limpios = limpiar_pares(pares)
#Guardar pares limpios en un archivo
guarda_datos_limpios(pares_limpios,'ingles-italiano-train.pkl')
guarda_datos_limpios(pares2,'ingles-test.pkl')
#Control al azar
for i in range(100):
    print('[%s] => [%s]'% (pares_limpios[i,0], pares_limpios[i,1]))

Saved: ingles-italiano-train.pkl
Saved: ingles-test.pkl
[we therefore respect whatever parliament may decide] => [quindi noi rispettiamo le eventuali decisioni in materia del parlamento]
[but i would also like to make it very clear that president prodi made a commitment to this parliament to introduce a new debate as mr baron crespo has reminded us which would be in addition to the annual debate on the commission s legislative programme on the broad areas of action for the next five years that is to say for this legislature] => [ma voglio dire altrettanto chiaramente che il presidente prodi si e impegnato con il parlamento a inserire un nuovo dibattito come ha ricordato l onorevole baron oltre al dibattito annuale sul programma legislativo della commissione sulle grandi linee di azione per il prossimo quinquennio cioe per la presente legislatura]
[madam president i would like to say that the agreement reached in september distinguished this debate from the annual presentation of the co

In [10]:
#DATOS DE PRUEBA:
#Cargar conjunto de datos de entrenamiento:
#texto_prueba = cargar_doc(r'C:\Users\fernando\Desktop\PLN Traductor\Final3\data3.eval')
#Dividirlo en parejas ingles-italiano:
#lista_prueba = crea_pares(texto)
#Limpiar esas oraciones:
#lista_limpios_prueba = limpiar_pares(lista_prueba)
#Guardar pares limpios en un archivo
#guarda_datos_limpios(lista_limpios_prueba,'ingles-prueba.pkl')

In [11]:
#Carga el conjunto de datos limpios gurdados:
dataset = cargar_datos_limpios('ingles-italiano-train.pkl')
#Test:
test_f = cargar_datos_limpios('ingles-test.pkl')
#Orden aleatorio
shuffle(dataset)
#Dividir en entrenamiento/prueba
train, test = dataset[:800], dataset[800:]
#Guardar conjuntos
#save_clean_data(dataset,'english-german-both.pkl')
guarda_datos_limpios(train,'ingles-italiano-train-f.pkl')
guarda_datos_limpios(test,'ingles-italiano-test.pkl')

Saved: ingles-italiano-train-f.pkl
Saved: ingles-italiano-test.pkl


In [12]:
#Cargar datos limpios y asignar variables:
#Carga el conjunto de datos limpios gurdados:
dataset = cargar_datos_limpios('ingles-italiano-train.pkl')
train = cargar_datos_limpios('ingles-italiano-train_f.pkl')
#Test:
#test_f = cargar_datos_limpios('ingles-test.pkl')
test = cargar_datos_limpios('ingles-italiano-test.pkl')

In [13]:
#TRATAR LOS DATOS QUE TENEMOS:

In [14]:
#Caben en un tokenizador
# la clase Keras Tokenize para mapear palabras a números enteros,
# según sea necesario para el modelado.
# La siguiente función create_tokenizer() entrenará un 
# tokenizer en una lista de frases.
# instalar un tokenizador:
def crear_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [15]:
#print(dataset[:, 0])
#ing = crear_tokenizer(dataset[:, 0])
#ing.word_index

#CREA UN DICCIONARIO CON LAS PALABRAS COMO KEYS Y UN NUMERO COMO VALUE:
#it = crear_tokenizer(dataset[:, 1])

#it.word_index
#dict.keys(it.word_index)
#for k,v in it.word_index.items():
#    print ("%s -> %s" %(k,v))

#DEL DICCIONARIO QUE SE CREA CON .WORD_INDEX ES PARA INVERTIR KEYS DE VALUES:
#Keys = dict.keys(it.word_index)
#Val = dict.values(it.word_index)
#dicList = dict(zip(Val,Keys))
#print(dicList)
#print(dicList.get(4279))
#print(it.word_index)

In [16]:
# Duración máxima de la frase:
# Econtrará la oración con el numero de palabras mayor y devuelve ese
#numero de palabras de esa oración:
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [17]:
#num = []
#for line in dataset[:, 0]:
#    num.append(len(line.split()))
    #print(line.split())
#print(max(num))
#max_length(dataset[:, 0])

In [18]:
# prepara el tokenizer en inglés
ingles_tokenizer = crear_tokenizer(dataset[:, 0])
ingles_vocab_size = len(ingles_tokenizer.word_index) + 1
ingles_length = max_length(dataset[:, 0])
print('Tamaño de Vocabulario Ingles: %d'% ingles_vocab_size)
print('Ingles Max Length: %d'% (ingles_length))
# Prepara el tokenizador en Italiano
italian_tokenizer = crear_tokenizer(dataset[:, 1])
italian_vocab_size = len(italian_tokenizer.word_index) + 1
italian_length = max_length(dataset[:, 1])
print('\nTamaño de Vocabulario Italiano: %d'% italian_vocab_size)
print('Italiano Max Lenthg: %d'% (italian_length))

Tamaño de Vocabulario Ingles: 3289
Ingles Max Length: 115

Tamaño de Vocabulario Italiano: 4419
Italiano Max Lenthg: 105


In [19]:
#Codificar:
#x = ingles_tokenizer.texts_to_sequences(train[:, 0])
#print(x[0])
#xp = pad_sequences(x, maxlen=ingles_length, padding='post')
#print(xp[0])

In [20]:
# Codificar y secuencias de pads 
#CODIFICA DANDO LISTAS CON EL VALOR DE CADA PALABRA EN EL TOKENIZER
#ES DECIR LA LISTA CONTIENE ORACIONES PERO LOS NUMEROS INDEXADOS CON 
#EL METODO TOKENIZER DE CADA PALABRA SON LOS QUE ESTAN EN CADA ELEMENTO DE
#LA LISTA. POR EJEMPLO SI EN EL DIC ESTA QUE = {HOLA:1, COMO:2}
#Y LA ORACION ES: HOLA COMO
#AL CODIFICAR SERIA [1 2]. ES COMO UN ENCODING Y CABE MENCIONAR QUE EL TAMAÑO
#DE ESA LISTA AL CODIFICAR SERA DEL TAMAÑO DE ORACIÓN MAS GRANDE QUE OBTUVIMOS
#DE MAX_LENGTH.
def encode_sequences(tokenizer, length, lines):
    # secuencias enteras de codificación
    #listas de oraciones donde cada elemento es el indice de cada palabra:
    X = tokenizer.texts_to_sequences(lines) 
    # secuencias de pads con valores 0 hasta el tamaño de 
    #la oracion mas grande, acompleta el vector con ceros
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

In [21]:
# Una secuencia de destino codificada en caliente
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [22]:
# preparar los datos de la formación
trainX = encode_sequences(ingles_tokenizer, ingles_length, dataset[:, 0])
trainY = encode_sequences(italian_tokenizer, italian_length, dataset[:, 1])
trainY = encode_output(trainY, italian_vocab_size)
# preparar los datos de validación
testX = encode_sequences(ingles_tokenizer, ingles_length, test[:, 0])
testY = encode_sequences(italian_tokenizer, italian_length, test[:, 1])
testY = encode_output(testY, italian_vocab_size)

In [23]:
print(trainX.shape)

(900, 115)


In [24]:
print(trainY.shape)

(900, 105, 4419)


In [25]:
#trainY = encode_output(trainY, ingles_vocab_size)

In [26]:
#print(trainY[0][0])

In [27]:
#clas = ingles_vocab_size
#print(clas)
#print(trainY)
#encode = to_categorical(trainY[0]) 
#enc = array(encode)
#en = enc.reshape(800)
#print(encode[0])

#l = []
#for sequence in trainY:
    #print(sequence)
#    encoded = to_categorical(sequence, num_classes = clas)
#    l.append(encoded)
#print(l[0])

In [28]:
#El vector se codifica en función del valor que se indexa en word_id tokenizer:
print(train[:, 1][0].split())
#print(len(train[:, 1][0].split()))

['e', 'noto', 'che', 'finora', 'il', 'consiglio', 'si', 'e', 'rifiutato', 'di', 'approvare', 'questa', 'nuova', 'normativa']


In [29]:
#print(trainX[0])

In [30]:
#print(trainY[0])

In [45]:
# definir el modelo NMT
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    # compilar modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    # resumir el modelo definido
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [46]:
#El modelo se entreno con 1000 epocas y 
# definir el modelo 
model = define_model(eng_vocab_size, ger_vocab_size, eng_length, ger_length, 256)
# modelo adecuado
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1,save_best_only=True, mode='min')
#model.fit(trainX, trainY, epochs=1000, batch_size=64, validation_data=(testX, testY),callbacks=[checkpoint], verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 115, 256)          841984    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 105, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 105, 256)          525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 105, 4419)         1135683   
Total params: 3,028,291
Trainable params: 3,028,291
Non-trainable params: 0
_________________________________________________________________


In [47]:
# asignar un número entero a una palabra
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [48]:
# generar la secuencia de origen del objetivo
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return' '.join(target)

In [49]:
# evaluar la habilidad del modelo
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources): 
        # traducir el texto fuente codificado
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, italian_tokenizer, source)
        raw_src, raw_target = raw_dataset[i]
        if i < 10:
            print('\nsrc=[%s], \ntarget=[%s], \npredicted=[%s]'% (raw_src, raw_target, translation))
            actual.append(raw_target.split())
            predicted.append(translation.split())
        # Calcular la puntuación BLEU
        #print('BLEU-1: %f'% corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
        #print('BLEU-2: %f'% corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
        #print('BLEU-3: %f'% corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
        #print('BLEU-4: %f'% corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25,0.25)))


In [60]:
# conjuntos de datos de carga
dataset = cargar_datos_limpios('ingles-italiano-train.pkl')
#train = cargar_datos_limpios('ingles-italiano-train.pkl')
test_f = cargar_datos_limpios('ingles-test.pkl')

# prepara el tokenizer en inglés
ingles_tokenizer = crear_tokenizer(dataset[:, 0])
ingles_vocab_size = len(ingles_tokenizer.word_index) + 1
ingles_length = max_length(dataset[:, 0])

# Prepara el tokenizador en Italiano
italian_tokenizer = crear_tokenizer(dataset[:, 1])
italian_vocab_size = len(italian_tokenizer.word_index) + 1
italian_length = max_length(dataset[:, 1])

# preparar los datos
trainX = encode_sequences(ingles_tokenizer, ingles_length, dataset[:, 0])
#testX = encode_sequences(italian_tokenizer, italian_length, test)

# modelo de carga
#model = load_model(r'C:\Users\fernando\Desktop\PLN Traductor\model.h5')
model = load_model('model.h5')
# probar con los datos de entrenamiento:
print('\ntrain')
evaluate_model(model, italian_tokenizer, trainX, dataset)
# probar en algunas secuencias de prueba
#print('\ntest')
#evaluate_model(model, ingles_tokenizer, testX, test)


train

src=[we therefore respect whatever parliament may decide], 
target=[quindi noi rispettiamo le eventuali decisioni in materia del parlamento], 
predicted=[quindi noi rispettiamo le eventuali decisioni in materia del parlamento]

src=[but i would also like to make it very clear that president prodi made a commitment to this parliament to introduce a new debate as mr baron crespo has reminded us which would be in addition to the annual debate on the commission s legislative programme on the broad areas of action for the next five years that is to say for this legislature], 
target=[ma voglio dire altrettanto chiaramente che il presidente prodi si e impegnato con il parlamento a inserire un nuovo dibattito come ha ricordato l onorevole baron oltre al dibattito annuale sul programma legislativo della commissione sulle grandi linee di azione per il prossimo quinquennio cioe per la presente legislatura], 
predicted=[ma voglio dire altrettanto altrettanto chiaramente il prodi prodi pro

In [64]:
#print(test[0])
test = cargar_datos_limpios('ingles-italiano-test.pkl')
testX = encode_sequences(ingles_tokenizer, ingles_length, test[:, 0])
print('\ntest')
evaluate_model(model, italian_tokenizer, testX, dataset)


test

src=[we therefore respect whatever parliament may decide], 
target=[quindi noi rispettiamo le eventuali decisioni in materia del parlamento], 
predicted=[quando domani approveremo il processo verbale verbale seduta odierna colleghi colleghi che dovessero ritenerlo inadeguato potranno chiedere che esso venga modificato]

src=[but i would also like to make it very clear that president prodi made a commitment to this parliament to introduce a new debate as mr baron crespo has reminded us which would be in addition to the annual debate on the commission s legislative programme on the broad areas of action for the next five years that is to say for this legislature], 
target=[ma voglio dire altrettanto chiaramente che il presidente prodi si e impegnato con il parlamento a inserire un nuovo dibattito come ha ricordato l onorevole baron oltre al dibattito annuale sul programma legislativo della commissione sulle grandi linee di azione per il prossimo quinquennio cioe per la presente le

In [89]:
# evaluar la habilidad del modelo
def evaluate_model_prueba(model, tokenizer, sources):
    predicted = list()
    for i, source in enumerate(sources): 
        # traducir el texto fuente codificado
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, italian_tokenizer, source)
        #raw_src, raw_target = raw_dataset[i]
        #print('\npredicted=[%s]'% (translation))
        #actual.append(raw_target.split())
        predicted.append(translation.split())
    print(predicted)
    #print(i)

In [95]:
prueba =['we therefore respect whatever parliament may decide']
prueba2 = encode_sequences(ingles_tokenizer, ingles_length, prueba)
#print(prueba2)
evaluate_model_prueba(model, italian_tokenizer, prueba2)

[['quindi', 'noi', 'rispettiamo', 'le', 'eventuali', 'decisioni', 'in', 'materia', 'del', 'parlamento']]


In [96]:
prueba = ['i was told that large sections of the socialist group were also keen to have this item taken off the agenda because at the vote in the conference of presidents no vote was received from the working group of members of the socialist group responsible for this matter']
prueba2 = encode_sequences(ingles_tokenizer, ingles_length, prueba)
evaluate_model_prueba(model, italian_tokenizer, prueba2)

[['mi', 'e', 'stato', 'detto', 'che', 'che', 'parte', 'cospicua', 'cospicua', 'gruppo', 'gruppo', 'socialista', 'vorrebbe', 'vorrebbe', 'venisse', 'venisse', 'venisse', 'ritirato', 'dallordine', 'dallordine', 'in', 'in', 'in', 'votazione', 'votazione', 'di', 'di', 'di', 'conferenza', 'presidenti', 'presidenti', 'mancato', 'mancato', 'voto', 'del', 'del', 'del', 'del', 'del', 'dei', 'competenti', 'competenti', 'del', 'gruppo', 'socialista']]


In [98]:
prueba =['we therefore respect whatever']
prueba2 = encode_sequences(ingles_tokenizer, ingles_length, prueba)
evaluate_model_prueba(model, italian_tokenizer, prueba2)

[['ci', 'sono', 'obiezioni']]
